In [0]:
Neo4jConnectionInfo = {
    "URI": "neo4j+s://62d77af8.databases.neo4j.io", 
    "Username": dbutils.secrets.get(scope="neo4j", key="username"), 
    "Password": dbutils.secrets.get(scope="neo4j", key="password")
}

LoadSettings = {
    "csvDir": "file:/Workspace/Users/eric.monk@neo4j.com/mind_csvs/",
    "writeMode": "Overwrite"
}

In [0]:
print(spark.sparkContext.defaultParallelism)
# spark.conf.get("spark.sparkContext.defaultParallelism")
# print(spark.conf.get("spark.default.parallelism"))
#print(spark.conf.get("spark.executor.cores"))
print(spark.conf.get("spark.executor.extraClassPath"))

256
/databricks/spark/dbconf/log4j/executor:/databricks/spark/dbconf/jets3t/:/databricks/spark/dbconf/hadoop:/databricks/hive/conf:/databricks/jars/*


In [0]:
from pyspark.sql.functions import substring, concat_ws

df = spark.read.table('historic_clicks')

# https://stackoverflow.com/questions/45512884/spark-dataframe-column-with-last-character-of-other-column
# newDf = df.withColumn('partitionCode', substring(df.userId, -1, 1))
newDf = df.withColumn('partitionCode', concat_ws('-', substring(df.userId, -1, 1), substring(df.newsId, -1, 1)))
pandasDf = newDf.toPandas()
grouped = pandasDf.groupby('partitionCode')


In [0]:
# df.show()
# df.columns
# newDf.show()
# print(grouped)
# grouped.describe()
print(grouped.groups.keys())
sparkDfs = {}

for key in grouped.groups.keys():
    segment = grouped.get_group(key)
    segment.describe()
    print(key + ': ' + str(len(segment.index)))

# for key in grouped.groups.keys():
#     sparkDfs[key] = spark.createDataFrame(grouped.get_group(key))

# print(sparkDfs)
#df00 = grouped.get_group('0-0')
#print(df00)
# df00.show()

dict_keys(['0', '0-0', '0-1', '0-2', '0-3', '0-4', '0-5', '0-6', '0-7', '0-8', '0-9', '1', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', '1-7', '1-8', '1-9', '2', '2-0', '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '3', '3-0', '3-1', '3-2', '3-3', '3-4', '3-5', '3-6', '3-7', '3-8', '3-9', '4', '4-0', '4-1', '4-2', '4-3', '4-4', '4-5', '4-6', '4-7', '4-8', '4-9', '5', '5-0', '5-1', '5-2', '5-3', '5-4', '5-5', '5-6', '5-7', '5-8', '5-9', '6', '6-0', '6-1', '6-2', '6-3', '6-4', '6-5', '6-6', '6-7', '6-8', '6-9', '7', '7-0', '7-1', '7-2', '7-3', '7-4', '7-5', '7-6', '7-7', '7-8', '7-9', '8', '8-0', '8-1', '8-2', '8-3', '8-4', '8-5', '8-6', '8-7', '8-8', '8-9', '9', '9-0', '9-1', '9-2', '9-3', '9-4', '9-5', '9-6', '9-7', '9-8', '9-9'])
0: 220
0-0: 21383
0-1: 21066
0-2: 19669
0-3: 20359
0-4: 20895
0-5: 20377
0-6: 22409
0-7: 20413
0-8: 21172
0-9: 21869
1: 211
1-0: 20636
1-1: 20120
1-2: 18611
1-3: 19713
1-4: 20498
1-5: 19936
1-6: 22195
1-7: 20076
1-8: 20301
1-9: 21227
2: 2

In [0]:
from pyspark.sql.functions import col

# Sample data
data = [("u11",), ("u21",), ("u31",), ("u41",), ("u51",),
        ("u12",), ("u13",), ("u22",), ("u23",), ("u53",), ("u43",), ("u32",)]

# Create DataFrame
df = spark.createDataFrame(data, ["userId"])

# Function to filter DataFrame by last digit of userId
def filter_by_last_digit(df, digit):
    return df.filter(col("userId").substr(-1, 1) == digit)

# Create separate DataFrames for each set of userIds with the same last digit
distinct_digits = df.select(df["userId"].substr(-1, 1)).distinct().collect()
dataframes = {}
for digit_row in distinct_digits:
    digit = digit_row[0]
    dataframes[digit] = filter_by_last_digit(df, digit)

# Print and show the DataFrames
for digit, dataframe in dataframes.items():
    print(f"DataFrame for userIds with last digit {digit}:")
    dataframe.show()


DataFrame for userIds with last digit 1:
+------+
|userId|
+------+
|   u11|
|   u21|
|   u31|
|   u41|
|   u51|
+------+

DataFrame for userIds with last digit 2:
+------+
|userId|
+------+
|   u12|
|   u22|
|   u32|
+------+

DataFrame for userIds with last digit 3:
+------+
|userId|
+------+
|   u13|
|   u23|
|   u53|
|   u43|
+------+



In [0]:

from pyspark.sql.functions import col, substring, concat_ws

# tableSize = 10 means a 10x10 table
def getPartitionsAndBatches (tableSize):
    batches = []
    for i in range(tableSize):
        partitions = []
        for j in range(tableSize):
            k = (i+j)%tableSize
            partitions.append(str(k) + '-' + str(j))
        batches.append(partitions)
    return batches

batches = getPartitionsAndBatches(10)
# batches = getPartitionsAndBatches(100)

# Function to filter DataFrame by last digit of userId
def filter_by_partition_code(df, colName, partitionSet):
    # print(colName, partitionSet)
    return df.filter(col(partitionColName).isin(partitionSet))

partitionColName = 'partitionCode'
# df = spark.read.table('historic_clicks')
df = spark.read.table('did_not_clicks')

# https://stackoverflow.com/questions/45512884/spark-dataframe-column-with-last-character-of-other-column
# newDf = df.withColumn('partitionCode', substring(df.userId, -1, 1))
newDf = df.withColumn(partitionColName, concat_ws('-', substring(df.userId, -1, 1), substring(df.newsId, -1, 1)))
# newDf = df.withColumn(partitionColName, concat_ws('-', substring(df.userId, -2, 2), substring(df.newsId, -2, 2)))
# newDf.show()

def writeDfToNeo (df, cypherQuery):
    result = (df.write
        .format("org.neo4j.spark.DataSource")
        .option("url", Neo4jConnectionInfo["URI"])
        .option("authentication.basic.username", Neo4jConnectionInfo["Username"])
        .option("authentication.basic.password", Neo4jConnectionInfo["Password"])
        .option("batch.size", 25000)
        .option("query", cypherQuery)
        .mode(LoadSettings["writeMode"])
        .save())


# Create separate DataFrames for each batch
dataframes = {}
for index, partitionSet in enumerate(batches):
    # print(partitionSet)
    dataframes[index] = filter_by_partition_code(newDf, partitionColName, partitionSet)
    # parallelRelDf = dataframes[index].repartition(partitionColName)
    parallelRelDf = dataframes[index].repartition(10, partitionColName)
    # parallelRelDf = dataframes[index].repartition(100, partitionColName)
    print('Num partitions (' + str(index) + '): ' + str(parallelRelDf.rdd.getNumPartitions()))

    pandasDf = parallelRelDf.toPandas()
    grouped = pandasDf.groupby(partitionColName)
    # grouped.describe()
    for key in grouped.groups.keys():
        segment = grouped.get_group(key)
        segment.describe()
        print(key + ': ' + str(len(segment.index)))    

    # parallelRelDf.show()
    # writeDfToNeo(parallelRelDf, """
    #         MATCH(user:User {userId: event.userId})
    #         MATCH(news:News {newsId: event.newsId})
    #         MERGE(user)-[r:HISTORICALLY_CLICKED {splitSet: event.splitSet}]->(news)
    #         RETURN count(r)
    # """)

    # writeDfToNeo(parallelRelDf, """
    #     MATCH(user:User {userId: event.userId})
    #     MATCH(news:News {newsId: event.newsId})
    #     MERGE(user)-[:DID_NOT_CLICK]->(news)
    # """)

        # bad - generates the wrong number of rels
        # WHERE NOT EXISTS{(user)-[:DID_NOT_CLICK]->(news)}
        # CREATE(user)-[r:DID_NOT_CLICK]->(news)

        # MERGE(user)-[r:DID_NOT_CLICK {
        #     splitSet: event.splitSet,
        #     impressionId: event.impressionId,
        #     impressionTime: event.time
        # }]->(news)


# Print and show the DataFrames
# for index, dataframe in dataframes.items():
#     print(f"DataFrame for index {index}:")
#     dataframe.show()



Num partitions (0): 10
0-0: 93906
1-1: 79137
2-2: 92085
3-3: 69603
4-4: 76708
5-5: 75705
6-6: 83277
7-7: 89520
8-8: 79672
9-9: 84095
Num partitions (1): 10
0-9: 85705
1-0: 94758
2-1: 78615
3-2: 91884
4-3: 70615
5-4: 73935
6-5: 76983
7-6: 85377
8-7: 84643
9-8: 81109
Num partitions (2): 10
0-8: 82172
1-9: 86080
2-0: 93423
3-1: 78441
4-2: 93274
5-3: 68244
6-4: 75390
7-5: 78887
8-6: 81025
9-7: 85658
Num partitions (3): 10
0-7: 87132
1-8: 82562
2-9: 85319
3-0: 93452
4-1: 80311
5-2: 90928
6-3: 70014
7-4: 77653
8-5: 74449
9-6: 82441
Num partitions (4): 10


In [0]:
from pyspark.sql.functions import col, substring, concat_ws

# tableSize = 10 means a 10x10 table
# df = spark.read.table('historic_clicks').repartition(1)
df = spark.read.table('did_not_clicks').repartition(1)

def writeDfToNeo (df, cypherQuery):
    result = (df.write
        .format("org.neo4j.spark.DataSource")
        .option("url", Neo4jConnectionInfo["URI"])
        .option("authentication.basic.username", Neo4jConnectionInfo["Username"])
        .option("authentication.basic.password", Neo4jConnectionInfo["Password"])
        .option("query", cypherQuery)
        .option("batch.size", 25000)
        .mode(LoadSettings["writeMode"])
        .save())


# writeDfToNeo(df, """
#         MATCH(user:User {userId: event.userId})
#         MATCH(news:News {newsId: event.newsId})
#         MERGE(user)-[r:HISTORICALLY_CLICKED {splitSet: event.splitSet}]->(news)
#         RETURN count(r)
# """)

writeDfToNeo(df, """
    MATCH(user:User {userId: event.userId})
    MATCH(news:News {newsId: event.newsId})
    MERGE(user)-[r:DID_NOT_CLICK {
        splitSet: event.splitSet,
        impressionId: event.impressionId,
        impressionTime: event.time
    }]->(news)
""")

    # MERGE(user)-[:DID_NOT_CLICK]->(news)




In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import pmod, col, substring, concat_ws


# df = spark.read.table('historic_clicks')
df = spark.read.table('did_not_clicks')
df = df.filter(df.newsId.isNotNull())

# https://stackoverflow.com/questions/45512884/spark-dataframe-column-with-last-character-of-other-column
partitionColName = 'partitionCode'
sortColName = 'sortVal'

sortedPartitionedDf = (df
    .withColumn(partitionColName, substring(df.newsId, -1, 1))    
    # .withColumn(sortColName, substring(df.userId, -4, 4))
    .withColumn(sortColName, concat_ws('',  \
                                    pmod(substring(df.userId, -1, 1).cast("int") +  \
                                    substring(df.newsId, -1, 1).cast("int"), 10).cast("string"), \
                                    substring(df.userId, -4, 3)))

    .orderBy(sortColName)
    .repartition(10, partitionColName))
    # .orderBy(sortColName, 'userId'))

sortedPartitionedDf.show()

writeDfToNeo(sortedPartitionedDf, """
        MATCH(user:User {userId: event.userId})
        MATCH(news:News {newsId: event.newsId})
        MERGE(user)-[:DID_NOT_CLICK]->(news)
""")

# MERGE(user)-[r:HISTORICALLY_CLICKED {splitSet: event.splitSet}]->(news)

+------------+------+-------------------+--------+------+-------------+-------+
|impressionId|userId|               time|splitSet|newsId|partitionCode|sortVal|
+------------+------+-------------------+--------+------+-------------+-------+
|      120614|U18360|2019-11-12 07:20:32|   TRAIN| N3737|            7|   7836|
|      120614|U18360|2019-11-12 07:20:32|   TRAIN|N28327|            7|   7836|
|      120614|U18360|2019-11-12 07:20:32|   TRAIN|N44397|            7|   7836|
|      120614|U18360|2019-11-12 07:20:32|   TRAIN|N34657|            7|   7836|
|      120615|U54668|2019-11-09 08:17:07|   TRAIN|N43377|            7|   5466|
|      120615|U54668|2019-11-09 08:17:07|   TRAIN|N54287|            7|   5466|
|      120615|U54668|2019-11-09 08:17:07|   TRAIN|N12977|            7|   5466|
|      120615|U54668|2019-11-09 08:17:07|   TRAIN|N53617|            7|   5466|
|      120615|U54668|2019-11-09 08:17:07|   TRAIN|N37377|            7|   5466|
|      120615|U54668|2019-11-09 08:17:07

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-395196616532658>, line 27
     23     # .orderBy(sortColName, 'userId'))
     25 sortedPartitionedDf.show()
---> 27 writeDfToNeo(sortedPartitionedDf, """
     28         MATCH(user:User {userId: event.userId})
     29         MATCH(news:News {newsId: event.newsId})
     30         MERGE(user)-[:DID_NOT_CLICK]->(news)
     31 """)
     33 # MERGE(user)-[r:HISTORICALLY_CLICKED {splitSet: event.splitSet}]->(news)

File <command-3170225426673041>, line 41, in writeDfToNeo(df, cypherQuery)
     32 def writeDfToNeo (df, cypherQuery):
     33     result = (df.write
     34         .format("org.neo4j.spark.DataSource")
     35         .option("url", Neo4jConnectionInfo["URI"])
     36         .option("authentication.basic.username", Neo4jConnectionInfo["Username"])
     37         .option("authentication.basic.password", Neo4jConn

In [0]:
myitems = []
for i in range(4):
    myset = []
    for j in range(4):
        k = (i+j)%4
        myset.append(str(k) + '_' + str(j))
    myitems.append(myset)

print(myitems)


[['0_0', '1_1', '2_2', '3_3'], ['1_0', '2_1', '3_2', '0_3'], ['2_0', '3_1', '0_2', '1_3']]


In [0]:
import math
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import pmod, col, count, substring, concat_ws, udf

# tableSize = 10 means a 10x10 table
def getPartitionsAndBatches (tableSize):
    batches = []
    for i in range(tableSize):
        partitions = []
        for j in range(tableSize):
            k = (i+j)%tableSize
            partitions.append(str(k) + '-' + str(j))
        batches.append(partitions)
    return batches

# batches = getPartitionsAndBatches(8)
batches = getPartitionsAndBatches(10)
# batches = getPartitionsAndBatches(100)

increment_udf = udf(lambda x: x + 1, IntegerType())
increment_udf2 = udf(lambda x, y: x + y, IntegerType())
# get_partition_code = udf(lambda x, y, z: str(math.floor((x or 0) / z)) + '-' + str(math.floor((y or 0) / z)), StringType())
get_partition_code_1 = udf(lambda x, y: str(math.floor((x or 0) / 1)) + '-' + str(math.floor((y or 0) / 1)), StringType())
get_partition_code_10 = udf(lambda x, y: str(math.floor((x or 0) / 10)) + '-' + str(math.floor((y or 0) / 10)), StringType())
get_partition_code_100 = udf(lambda x, y: str(math.floor((x or 0) / 100)) + '-' + str(math.floor((y or 0) / 100)), StringType())
# get_partition_code = udf(lambda x, y: math.floor(x / 25), IntegerType())

# Function to filter DataFrame by last digit of userId
def filter_by_partition_code(df, colName, partitionSet):
    # print(colName, partitionSet)
    return df.filter(col(partitionColName).isin(partitionSet))

partitionColName = 'partitionCode'
partitionColName2 = 'partitionCode2'
df = spark.read.table('historic_clicks')
# df = spark.read.table('did_not_clicks')

# https://stackoverflow.com/questions/45512884/spark-dataframe-column-with-last-character-of-other-column
# newDf = df.withColumn('partitionCode', substring(df.userId, -1, 1))
# newDf = df.withColumn(partitionColName, concat_ws('-', substring(df.userId, -1, 1), substring(df.newsId, -1, 1)))
# newDf = df.withColumn(partitionColName, concat_ws('-', substring(df.userId, -2, 2), substring(df.newsId, -2, 2)))
# newDf = df.withColumn(partitionColName, \
#                       get_partition_code(substring(df.userId, -2, 2).cast("int"), \
#                                      substring(df.newsId, -2, 2).cast("int")) \
#                       )

newDf1 = df.withColumn(partitionColName, \
                      get_partition_code_1(substring(df.userId, -1, 1).cast("int"), \
                                     substring(df.newsId, -1, 1).cast("int")) \
                      ) 

# newDf2 = df.withColumn(partitionColName, \
#                       get_partition_code_10(substring(df.userId, -2, 2).cast("int"), \
#                                      substring(df.newsId, -2, 2).cast("int")) \
#                       )                       

# newDf3 = df.withColumn(partitionColName, \
#                       get_partition_code_100(substring(df.userId, -3, 3).cast("int"), \
#                                      substring(df.newsId, -3, 3).cast("int")) \
#                       ) 

# newDf.show()

newDf = newDf1

# print()
# print('Result 1')
# countResult = newDf1.groupBy(partitionColName).count()
# pandasResult = countResult.toPandas()
# print(pandasResult.to_string())

# print()
# print('Result 10')
# countResult = newDf2.groupBy(partitionColName).count()
# pandasResult = countResult.toPandas()
# print(pandasResult.to_string())

# print()
# print('Result 100')
# countResult = newDf3.groupBy(partitionColName).count()
# pandasResult = countResult.toPandas()
# print(pandasResult.to_string())


def writeDfToNeo (df, cypherQuery):
    result = (df.write
        .format("org.neo4j.spark.DataSource")
        .option("url", Neo4jConnectionInfo["URI"])
        .option("authentication.basic.username", Neo4jConnectionInfo["Username"])
        .option("authentication.basic.password", Neo4jConnectionInfo["Password"])
        .option("batch.size", 25000)
        .option("query", cypherQuery)
        .mode(LoadSettings["writeMode"])
        .save())


# Create separate DataFrames for each batch
dataframes = {}
for index, partitionSet in enumerate(batches):
    print(partitionSet)
    dataframes[index] = filter_by_partition_code(newDf, partitionColName, partitionSet)
    # parallelRelDf = dataframes[index].repartition(partitionColName)
    # parallelRelDf = dataframes[index].repartition(5, partitionColName)
    # parallelRelDf = dataframes[index].repartition(8, partitionColName)
    parallelRelDf = dataframes[index].repartition(10, partitionColName)
    # parallelRelDf = dataframes[index].repartition(20, partitionColName)
    # parallelRelDf = dataframes[index].repartition(100, partitionColName)
    print('Num partitions (' + str(index) + '): ' + str(parallelRelDf.rdd.getNumPartitions()))

    # pandasDf = parallelRelDf.toPandas()
    # grouped = pandasDf.groupby(partitionColName)
    # # grouped.describe()
    # for key in grouped.groups.keys():
    #     segment = grouped.get_group(key)
    #     segment.describe()
    #     print(key + ': ' + str(len(segment.index)))    

    # parallelRelDf.show()
    writeDfToNeo(parallelRelDf, """
            MATCH(user:User {userId: event.userId})
            MATCH(news:News {newsId: event.newsId})
            MERGE(user)-[r:HISTORICALLY_CLICKED {splitSet: event.splitSet}]->(news)
    """)

    # writeDfToNeo(parallelRelDf, """
    #     MATCH(user:User {userId: event.userId})
    #     MATCH(news:News {newsId: event.newsId})
    #     MERGE(user)-[:DID_NOT_CLICK]->(news)
    # """)

    # writeDfToNeo(parallelRelDf, """
    #     MATCH(user:User {userId: event.userId})
    #     MATCH(news:News {newsId: event.newsId})
    #     MERGE(user)-[r:DID_NOT_CLICK {
    #         splitSet: event.splitSet,
    #         impressionId: event.impressionId,
    #         impressionTime: event.time
    #     }]->(news)
    # """)




['0-0', '1-1', '2-2', '3-3', '4-4', '5-5', '6-6', '7-7', '8-8', '9-9']
Num partitions (0): 10
['1-0', '2-1', '3-2', '4-3', '5-4', '6-5', '7-6', '8-7', '9-8', '0-9']
Num partitions (1): 10
['2-0', '3-1', '4-2', '5-3', '6-4', '7-5', '8-6', '9-7', '0-8', '1-9']
Num partitions (2): 10
['3-0', '4-1', '5-2', '6-3', '7-4', '8-5', '9-6', '0-7', '1-8', '2-9']
Num partitions (3): 10
['4-0', '5-1', '6-2', '7-3', '8-4', '9-5', '0-6', '1-7', '2-8', '3-9']
Num partitions (4): 10
['5-0', '6-1', '7-2', '8-3', '9-4', '0-5', '1-6', '2-7', '3-8', '4-9']
Num partitions (5): 10
['6-0', '7-1', '8-2', '9-3', '0-4', '1-5', '2-6', '3-7', '4-8', '5-9']
Num partitions (6): 10
['7-0', '8-1', '9-2', '0-3', '1-4', '2-5', '3-6', '4-7', '5-8', '6-9']
Num partitions (7): 10
['8-0', '9-1', '0-2', '1-3', '2-4', '3-5', '4-6', '5-7', '6-8', '7-9']
Num partitions (8): 10
['9-0', '0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-7', '7-8', '8-9']
Num partitions (9): 10
